In [1]:
%matplotlib inline
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from astropy.io import fits
import richardsplot
import pandas as pd
#from densityplot import *

In [2]:
def figsize(hscale, 
            vscale=(np.sqrt(5.0)-1.0)/2.0,
            fig_width_pt = 336.0):
   

    """
    Get the fig_width_pt by inserting \the\textwidth into LaTeX document.

    hscale is fraction of text width you want.

    vscale is fraction of hscale (defaults to golden ratio)  
    """
   
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    fig_width = fig_width_pt*inches_per_pt*hscale   # width in inches
    fig_height = fig_width*vscale                   # height in inches
    fig_size = [fig_width, fig_height]

    return fig_size

In [3]:
import palettable

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "axes.linewidth":1.5,               # width of box, 2 is too wide, 1 is too narrow
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 16,               # LaTeX default is 10pt font, font size of axis text label
    "axes.labelpad" : 6,                # Distance between label and axis
    "axes.formatter.limits":[-5, 5],    # use sci notation if log10 of axis range is smaller than first or larger than second 
    "axes.formatter.useoffset":False,
    "font.size": 16,
    "legend.fontsize": 12,              # Make the legend/label fonts a little smaller
    "xtick.labelsize": 16,              # Font size of numbers 
    "ytick.labelsize": 16,
    'xtick.major.width':1, 
    'xtick.minor.width':1, 
    'ytick.major.width':1, 
    'ytick.minor.width':1, 
    'xtick.major.size':10,             # size of tickmarks in points
    'xtick.minor.size':5, 
    'ytick.major.size':10, 
    'ytick.minor.size':5,
    'xtick.major.pad':8,               # distance between box and numbers
    'ytick.major.pad':8,
    'xtick.direction':'in',
    'ytick.direction':'in',
    'figure.autolayout': True,
    "figure.figsize": figsize(1,1),     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it
        r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
        ]
    }

mpl.rcParams.update(pgf_with_latex)


csdark = palettable.colorbrewer.qualitative.Dark2_5.mpl_colors
cspurple = palettable.colorbrewer.sequential.BuPu_4.mpl_colors
csorange = palettable.colorbrewer.sequential.YlOrBr_5.mpl_colors


In [4]:
#import pandas as pd
#Load the ICA weights from Paul's 6-component fitting (now for DR7)
infile = 'bokeh_files/grdr7_180126.weights'
infoHewett = pd.read_csv(infile, sep = ' ', names=["name", "redshift", "W1", "W2", "W3", "W4", "W5", "W6", "detection"]) #added detection for bokeh plot
infile1 = 'bokeh_files/grdr7.radec'
infoCoords = pd.read_csv(infile1, sep = ' ', names=["name", "RA", "Dec"])

dMask=np.array(infoHewett["detection"])
ih = np.array(infoHewett)

for i in ih[dMask==1]:
    print(i[0])
#print infoCoords

FileNotFoundError: File b'bokeh_files/grdr7_180126.weights' does not exist

In [ ]:

infile3 = 'grdr7_Shen2011_targ_uni.csv'
#infile4 = 'gr_info_a_short_coords_Allen2011.csv'

infoShen = pd.read_csv(infile3, skiprows=1, names=["name","RAx","DECx","SDSSNAME","RA","DEC","redshift","TARG_FLAG","UNI_FLAG","COLOR_FLAG","MIZ2","BALFLAGShen","FIRSTFRTYPE","FINTREST6CM","LOGFNU2500","R6CM2500A","LOGL3000","LOGL1350","Separation", "DETECTION"])
#infoAllen = pd.read_csv(infile4, skiprows=1, names=["file","name","redshift","BALflag","RAx","DECx","SDSS","RAdeg","DEdeg","z","imag","SN","logF1700","logL1700","C4BI","C4d","C4Vmin","C4VMax","Separation"])

#print infoShen['redshift']

In [ ]:
#Mask to pick out color-selected objects (ignores radio selected and NaN [non-DR7 objects])
cMask = np.array(infoShen['COLOR_FLAG'])

#This mask picks out our detections
dMask = np.array(infoShen['DETECTION'])

In [ ]:
#Need to format weights for Scikit-Learn
weights = infoHewett[infoHewett.columns[2:]]
#print weights
X = np.array(weights)
print(X)

wt_dMask = np.array(infoHewett["detection"])

wt_dX = X[wt_dMask==1] #find detection entries
wt_dX = wt_dX[:,:6] #reformat weights (eliminate detection column)
#print(wt_dX)

wt_ndX = X[wt_dMask==-1]
wt_ndX = wt_ndX[:,:6]
#print(wt_ndX)

 
    
# Array of each of the 6 weights
W1 = X[:,0]
W2 = X[:,1]
W3 = X[:,2]
W4 = X[:,3]
W5 = X[:,4]
W6 = X[:,5]

W0 = X[:,0]+X[:,1]+X[:,2]+X[:,3]

In [ ]:
plt.figure(figsize=(8,8))
plt.hist(infoHewett['redshift'], color='b', bins=50, histtype='step')
plt.xlim(1.4,2.5)

In [ ]:
#We are going to want to restrict the redshift range, do it like this.
zem = np.array(infoHewett['redshift'])

# Number with z<1.75
zmask = ((zem<=1.75))
print (len(infoHewett[zmask]), len(infoHewett))
print (len(X[zmask]))
print (zem)

In [ ]:
#Distribution of quasars in ICA weight space
plt.figure(figsize=(9,8))

plt.scatter(W1, W2, c=W3, cmap="nipy_spectral", edgecolor="None")
plt.xlabel('W1')
plt.ylabel('W2')
plt.xlim(0,0.07)
plt.ylim(0,0.06)
cbar = plt.colorbar()
cbar.ax.set_ylabel('W3')

In [ ]:
#GTR: Did this a different way below.
#np.savetxt("grdr7.tSNE2.csv", projTSNE2, delimiter=",")

In [ ]:
# t-SNE
from sklearn.manifold import TSNE
tsne2 = TSNE(n_components = 2)
projTSNE2 = tsne2.fit_transform(X[:,:6])

#dproj = tsne2.fit_transform(wt_dX)
#ndproj = tsne2.fit_transform(wt_ndX)

In [ ]:
dproj = projTSNE2[wt_dMask==1] #organize coordinates to match d/nd in bokeh plot
ndproj = projTSNE2[wt_dMask==-1]

In [ ]:
print(projTSNE2)

In [ ]:
#Output tSNE projections to file
#infile1 = 'grdr7.radec'
#infoCoords = pd.read_csv(infile1, sep = ' ', names=["name", "RA", "Dec"])

df = pd.DataFrame({'Name':infoCoords['name'],'RA':infoCoords['RA'],'Dec':infoCoords['Dec'], 'Proj1':projTSNE2[:,0], 'Proj2':-1.0*projTSNE2[:,1]})
df.to_csv('grdr7.radec.projTSNE.csv')


In [ ]:
plt.figure(figsize=(25,5))
plt.subplot(1, 4, 1)
plt.scatter(projTSNE2[:,0], projTSNE2[:,1], c=W1, cmap="nipy_spectral", edgecolor="None")
plt.xlabel("Proj1")
plt.ylabel("Proj2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('W1')
plt.subplot(1, 4, 2)
plt.scatter(projTSNE2[:,0], projTSNE2[:,1], c=W2, cmap="nipy_spectral", edgecolor="None")
plt.xlabel("Proj1")
plt.ylabel("Proj2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('W2')
plt.subplot(1, 4, 3)
plt.scatter(projTSNE2[:,0], projTSNE2[:,1], c=W3, cmap="nipy_spectral", edgecolor="None")
plt.xlabel("Proj1")
plt.ylabel("Proj2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('W3')
plt.subplot(1, 4, 4)
plt.scatter(projTSNE2[:,0], projTSNE2[:,1], c=W4, cmap="nipy_spectral", edgecolor="None")
plt.xlabel("Proj1")
plt.ylabel("Proj2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('W4')
plt.tight_layout()

In [ ]:
plt.figure(figsize=(9,8))
plt.scatter(projTSNE2[:,0], projTSNE2[:,1], c=W1, cmap="nipy_spectral", edgecolor="None")
plt.xlabel("Proj1")
plt.ylabel("Proj2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('W1')
plt.xlim(-90,90)
plt.ylim(-90,90)


# Find radius of circle containing 20% of the sources and plot
x0 = 0 #take center of circle to be 0,0
y0 = 0
x  = projTSNE2[:,0]
y  = projTSNE2[:,1]
# Find radial coordinate of each point
r  = np.sqrt((x - x0)**2 + (y - y0)**2)
t  = 20 # percent
# Find radius that corresponds to t percentile
r0 = np.percentile(r, t)

mask0 = (r <= r0)
maskI = ((r>r0)&(x>0)&(y>0))
maskII = ((r>r0)&(x<=0)&(y>0))
maskIII = ((r>r0)&(x<=0)&(y<=0))
maskIV = ((r>r0)&(x>0)&(y<=0))

# Count the number of objects
#n_0 = (r <= r0).sum()
n_0 = mask0.sum()
print (r0, n_0)

circle=plt.Circle((0, 0), r0, color='k', fill=False)
fig = plt.gcf()
fig.gca().add_artist(circle)

plt.plot([0,0], [r0,100], c='k')
plt.plot([0,0], [-r0,-100], c='k')
plt.plot([r0,100], [0,0], c='k')
plt.plot([-r0,-100], [0,0], c='k')

n_I = maskI.sum()
n_II = maskII.sum()
n_III = maskIII.sum()
n_IV = maskIV.sum()
print (n_0,n_I, n_II, n_III, n_IV)

In [ ]:
#Read in radio data
R = np.array(infoShen['R6CM2500A']).astype(float)
FIRST = np.array(infoShen['FINTREST6CM']).astype(float)

In [ ]:
#Radio-loud mask
RLmask = (R>10)

In [ ]:
projTSNE2rad = projTSNE2[RLmask]
print (len(projTSNE2), len(projTSNE2rad))

In [ ]:
plt.figure(figsize=(9,8))

plt.scatter(projTSNE2[:,0], projTSNE2[:,1], c=W1, cmap="nipy_spectral", edgecolor="None", s=20)
plt.xlabel("Proj1")
plt.ylabel("Proj2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('W1')
plt.scatter(projTSNE2rad[:,0], projTSNE2rad[:,1], c='w', edgecolor='k', s=40)
plt.xlabel("Proj1")
plt.xlim(-90,90)
plt.ylim(-90,90)

circle=plt.Circle((0, 0), r0, color='k', fill=False)
fig = plt.gcf()
fig.gca().add_artist(circle)

plt.plot([0,0], [r0,100], c='k')
plt.plot([0,0], [-r0,-100], c='k')
plt.plot([r0,100], [0,0], c='k')
plt.plot([-r0,-100], [0,0], c='k')

Compute the RL fractions

In [ ]:
# Find radius of circle containing 20% of the sources and plot
x0 = 0 #take center of circle to be 0,0
y0 = 0
xRL  = projTSNE2rad[:,0]
yRL  = projTSNE2rad[:,1]
# Find radial coordinate of each point
rRL  = np.sqrt((xRL - x0)**2 + (yRL - y0)**2)
#t  = 20 # percent
# Find radius that corresponds to t percentile
#r0 = np.percentile(r, t)

mask0RL = (rRL <= r0)
maskIRL = ((rRL>r0)&(xRL>0)&(yRL>0))
maskIIRL = ((rRL>r0)&(xRL<=0)&(yRL>0))
maskIIIRL = ((rRL>r0)&(xRL<=0)&(yRL<=0))
maskIVRL = ((rRL>r0)&(xRL>0)&(yRL<=0))

# Count the number of objects
#n_0 = (r <= r0).sum()
n_0RL = mask0RL.sum()

n_IRL = maskIRL.sum()
n_IIRL = maskIIRL.sum()
n_IIIRL = maskIIIRL.sum()
n_IVRL = maskIVRL.sum()
print (n_0RL,n_IRL, n_IIRL, n_IIIRL, n_IVRL)
 
    
coreRLF = 100.0*n_0RL/n_0
quadIRLF = 100.0*n_IRL/n_I
quadIIRLF = 100.0*n_IIRL/n_II
quadIIIRLF = 100.0*n_IIIRL/n_III
quadIVRLF = 100.0*n_IVRL/n_IV
print ("Core RLF = %.2f percent" % coreRLF)
print ("Quad I RLF = %.2f percent" % quadIRLF)
print ("Quad II RLF = %.2f percent" % quadIIRLF)
print ("Quad III RLF = %.2f percent" % quadIIIRLF)
print ("Quad IV RLF = %.2f percent" % quadIVRLF)

Same plot as above, but now with color-selection targeting mask, and limited to $1.55<z<1.65$ (large black circles).

In [ ]:
#We are going to want to restrict the redshift range, do it like this.
zem = np.array(infoHewett['redshift'])

# Number with 1.55<z.1.75
#zmask = ((zem>1.65)&(zem<=1.658))
zmask = ((zem>1.645)&(zem<=1.6519))
print (len(infoHewett[zmask]), len(infoHewett))
print (len(X[zmask]))

#print (infoHewett[zmask])

Two of these 50 are FIRST sources, 4 are BALs.

2353 and 1019

In [ ]:
plot = plt.figure(figsize=(10,8))

projTSNE2col = projTSNE2[cMask>0]
projTSNE2radcol = projTSNE2[(cMask>0)&(RLmask)]
projTSNE2colz = projTSNE2[(cMask>0)&(zmask)]
print (len(projTSNE2colz))

#Arbitrarily flip the signs of the y-axis projection so that the quadrants align with the CIV plot
#plt.scatter(projTSNE2col[:,0], -1.0*projTSNE2col[:,1], c=W1[cMask>0], cmap="nipy_spectral", edgecolor="None", s=20)
plt.scatter(projTSNE2col[:,0], -1.0*projTSNE2col[:,1], c=W1[cMask>0], cmap="PRGn", edgecolor="None", s=20)
#plt.scatter(projTSNE2col[:,0], -1.0*projTSNE2col[:,1], c=W1[cMask>0], cmap="RdYlBu", edgecolor="None", s=20)
plt.xlabel("Proj1")
plt.ylabel("Proj2")
cbar = plt.colorbar()
cbar.ax.set_ylabel('W1')
plt.scatter(projTSNE2radcol[:,0], -1.0*projTSNE2radcol[:,1], c='w', edgecolor='k', s=60, label="FIRST")
plt.scatter(projTSNE2colz[:,0], -1.0*projTSNE2colz[:,1], c='k', edgecolor='w', s=500, marker='*', label="Targets")
plt.xlabel("Proj1")
plt.xlim(-90,90)
plt.ylim(-90,90)
plt.legend(loc="upper left")

circle=plt.Circle((0, 0), r0, color='k', fill=False, linewidth=2)
fig = plt.gcf()
fig.gca().add_artist(circle)

plt.plot([0,0], [r0,100], c='k', linewidth=2)
plt.plot([0,0], [-r0,-100], c='k', linewidth=2)
plt.plot([r0,100], [0,0], c='k', linewidth=2)
plt.plot([-r0,-100], [0,0], c='k', linewidth=2)
plt.savefig('newPGtargets.pdf')

In [ ]:
def make_sameName(i): #remove necessary characters to match image formatting (only to arcsec, remove SDSS)
    
    i = list(i)
    i.pop(0)
    i.pop(0)
    i.pop(0)
    i.pop(0)
    i.pop(5)
    i.pop(5)
    i.pop(5)
    i.pop(5)
    i.pop(5)
    i.pop(10)
    i.pop(10)
    i.pop(10)
    i.pop(10)

    i = ''.join(i)

    return i


Create Interactive Bokeh Plot of Data Above:

J1019+4948 is labelled as a target in this but was not observed (not in OPT)

In [ ]:
import bokeh.palettes as pal
from bokeh.plotting import figure, ColumnDataSource, output_file, show, output_notebook
from bokeh.models import HoverTool

output_file('interactive_VLA2018b.html')


dhover = HoverTool(names = ["Detections"], tooltips='''
    <div>
        <div>
            <img
                src="@imgs_4p5am_detection" height="250" alt="@imgs_4p5am_detection" width="250"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
            
            <img       
                src="@imgs_90as_detection" height="250" alt="@imgs_90as_detection" width="250"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div> 
          
        <div>
            <span style="font-size: 17px; font-weight: bold;">@obs_detection</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
    </div>


                      ''')

ndhover = HoverTool(names = ["NonDetections"], tooltips='''
    <div>
        <div>
            <img
                src="@imgs_4p5am_nondetection" height="250" alt="@imgs_4p5am_nondetection" width="250"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
            
            <img       
                src="@imgs_90as_nondetection" height="250" alt="@imgs_90as_nondetection" width="250"
                style="float: left; margin: 0px 15px 15px 0px;"
                border="2"
            ></img>
        </div> 
          
        <div>
            <span style="font-size: 17px; font-weight: bold;">@obs_nondetection</span>
            <span style="font-size: 15px; color: #966;">[$index]</span>
        </div>
        <div>
            <span style="font-size: 15px;">Location</span>
            <span style="font-size: 10px; color: #696;">($x, $y)</span>
        </div>
    </div>


                      ''')

detection_data = np.array(infoHewett[dMask==1])  #make array for all data of detections

detection_names = [] #make new list to iterate through all obs names
for i in detection_data:
    detection_names.append(i[0])
detection_same_names = [make_sameName(x) for x in detection_names] #create list with new names titled JXXXX+XXXX

detection_source = ColumnDataSource(data=dict(  #create source for detections
    
    x = dproj[:,0], 
    y = -1.0*dproj[:,1], #flip y-axis
    obs_detection = detection_same_names,
    imgs_4p5am_detection = [('bokeh_files/4p5am/Detections/' + name + '_4p5am.png') for name in detection_same_names], 
    imgs_90as_detection = [('bokeh_files/90as/Detections/' + name + '_90as.png') for name in detection_same_names]
    
))


#Now make another source for the non-detections
nondetection_data = np.array(infoHewett[dMask==-1])  

nondetection_names = [] #make new list to iterate through all obs names
for i in nondetection_data:
    nondetection_names.append(i[0])
nondetection_same_names = [make_sameName(x) for x in nondetection_names] #create list with new names titled JXXXX+XXXX

nondetection_source = ColumnDataSource(data=dict(  #create source for detections
    
    x = ndproj[:,0], 
    y = -1.0*ndproj[:,1], #flip y-axis
    obs_nondetection = nondetection_same_names,
    imgs_4p5am_nondetection = [('bokeh_files/4p5am/NonDetections/' + name + '_4p5am.png') for name in nondetection_same_names], 
    imgs_90as_nondetection = [('bokeh_files/90as/NonDetections/' + name + '_90as.png') for name in nondetection_same_names]
    
))



###

cmap = pal.brewer['PRGn'][11] 
sf = 11/.075 #scale position in color map to fit an index

p = figure(plot_width=800, plot_height=800, title="",
           x_range=[-100, 100], y_range = [-100, 100], tools=[dhover, ndhover, 'pan,box_zoom,wheel_zoom,save,reset,help'])

p.legend.location = 'top_right'

W1_cMask = W1[cMask>0]
for i in range(len(W1_cMask)): #loop through each weight and determine color based on weight
    p.circle(projTSNE2col[i,0], -1.0*projTSNE2col[i,1], color=cmap[10-int(W1_cMask[i]*sf)], name = "Background") #plot background sources (cmap inverted to match original)

p.circle(projTSNE2radcol[:,0], -1.0*projTSNE2radcol[:,1], line_color = 'black', fill_color = 'white', 
         size = 10, legend = 'FIRST') #FIRST sources

#FIRST and background are OK -- fix projections for targets

p.diamond_cross(x='x', y='y', color = 'red', size = 15, 
                source=detection_source, name = "Detections", legend = 'Detections') #Detections

p.diamond_cross(x='x', y='y', color = 'black', size = 15, 
                source=nondetection_source, name = "NonDetections", legend = 'Non-Detections') #NonDetections


show(p)

In [ ]:
print(projTSNE2colz)

In [ ]:
projTSNE2colz0 = projTSNE2[(cMask>0)&(zmask)&mask0]
projTSNE2colzI = projTSNE2[(cMask>0)&(zmask)&maskI]
projTSNE2colzII = projTSNE2[(cMask>0)&(zmask)&maskII]
projTSNE2colzIII = projTSNE2[(cMask>0)&(zmask)&maskIII]
projTSNE2colzIV = projTSNE2[(cMask>0)&(zmask)&maskIV]

print (len(projTSNE2colz0), len(projTSNE2colzI), len(projTSNE2colzII), len(projTSNE2colzIII), len(projTSNE2colzIV))


Write out arrays with these objects

In [ ]:
names=["index","name", "redshift", "W1", "W2", "W3", "W4", "W5", "W6"]
dftest = pd.DataFrame(infoHewett, columns=names)
dftest.to_csv("test.csv")